# Folium Demo - Interactive Maps

This notebook demonstrates how to use folium to create interactive maps for weather data visualization.
Note: We switched from pyleaflet to folium because pyleaflet has limited functionality.


In [1]:
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

print("Folium version:", folium.__version__)
print("Ready to create interactive maps!")


Folium version: 0.20.0
Ready to create interactive maps!


## Basic Map Creation


In [2]:

# Create a basic map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=10)
# Change center location from NYC to Chicago
m = folium.Map(location=[41.8781, -87.6298], zoom_start=10)

# Add a marker for NYC
# folium.Marker([40.7128, -74.0060], popup="New York City").add_to(m)

# Display the map
m


## Multiple Markers and Custom Icons


In [3]:
# Create a map with multiple weather stations
m2 = folium.Map(location=[39.8283, -98.5795], zoom_start=4)  # Center on USA

# Sample weather station data
stations = [
    {"name": "New York", "lat": 40.7128, "lon": -74.0060, "temp": 15.5},
    {"name": "Los Angeles", "lat": 34.0522, "lon": -118.2437, "temp": 22.3},
    {"name": "Chicago", "lat": 41.8781, "lon": -87.6298, "temp": 8.7},
    {"name": "Miami", "lat": 25.7617, "lon": -80.1918, "temp": 28.9},
    {"name": "Seattle", "lat": 47.6062, "lon": -122.3321, "temp": 12.1}
]

# Add markers for each station
for station in stations:
    popup_text = f"""
    <b>{station['name']}</b><br>
    Temperature: {station['temp']}°C<br>
    Coordinates: {station['lat']:.4f}, {station['lon']:.4f}
    """
    folium.Marker([station['lat'], station['lon']], popup=popup_text).add_to(m2)

m2


## Color-Coded Markers by Temperature


In [4]:
# Create a map with color-coded temperature markers
m3 = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

def get_temp_color(temp):
    """Get color based on temperature"""
    if temp < 10:
        return 'blue'
    elif temp < 20:
        return 'green'
    elif temp < 30:
        return 'orange'
    else:
        return 'red'

# Add color-coded markers
for station in stations:
    color = get_temp_color(station['temp'])
    popup_text = f"""
    <div style='text-align: center;'>
        <h3 style='color: {color};'>{station['name']}</h3>
        <p><strong>Temperature:</strong> {station['temp']}°C</p>
        <p><strong>Status:</strong> {'Hot' if station['temp'] > 25 else 'Cool' if station['temp'] < 15 else 'Moderate'}</p>
    </div>
    """
    folium.Marker([station['lat'], station['lon']], 
                  popup=popup_text, 
                  icon=folium.Icon(color=color)).add_to(m3)

m3


## Interactive Heatmap


In [5]:
# Create a heatmap of temperature data
m4 = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Generate more data points for a better heatmap
np.random.seed(42)
n_points = 50
lat_range = (25, 50)
lon_range = (-125, -65)

# Generate random points with temperature data
heatmap_data = []
for i in range(n_points):
    lat = np.random.uniform(lat_range[0], lat_range[1])
    lon = np.random.uniform(lon_range[0], lon_range[1])
    # Temperature varies with latitude (colder in north)
    temp = 30 - (lat - 25) * 0.5 + np.random.normal(0, 5)
    heatmap_data.append([lat, lon, temp])

# Add heatmap layer using folium's HeatMap
from folium.plugins import HeatMap
HeatMap(heatmap_data, radius=20, blur=15, max_zoom=1).add_to(m4)

# Add some markers for reference
for i, (lat, lon, temp) in enumerate(heatmap_data[::10]):  # Every 10th point
    folium.Marker([lat, lon], popup=f"Temp: {temp:.1f}°C").add_to(m4)

m4


## Map Layers and Controls


In [6]:
# Create a map with multiple layers and controls
m5 = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Add different base layers (folium uses different approach)
# Note: Folium handles base layers differently than pyleaflet

# Add circles to show temperature range
for station in stations:
    # Circle size based on temperature
    radius = station['temp'] * 1000  # Scale for visibility
    
    # Add circle using folium
    folium.CircleMarker([station['lat'], station['lon']], 
                       radius=radius/1000,  # Scale down for folium
                       color=get_temp_color(station['temp']),
                       fill=True,
                       popup=f"{station['name']}: {station['temp']}°C").add_to(m5)

# Add a rectangle for a region of interest
folium.Rectangle([[35, -100], [45, -80]], 
                color='red', 
                fill=False, 
                popup="Analysis Region").add_to(m5)

m5


## Real Weather Data Integration


In [7]:
# Load real weather station data and create an interactive map
try:
    import dask.dataframe as dd
    
    # Load station data
    stations_df = dd.read_parquet("../weather_info/stations_dask.parquet")
    stations_sample = stations_df.head(100).compute()  # Get first 100 stations
    
    # Create map
    m6 = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
    
    # Add markers for real weather stations
    for idx, station in stations_sample.iterrows():
        popup_text = f"""
        <div style='text-align: center;'>
            <h4>{station['name']}</h4>
            <p><strong>Station ID:</strong> {station['station_id']}</p>
            <p><strong>Location:</strong> {station['latitude']:.4f}, {station['longitude']:.4f}</p>
            <p><strong>Elevation:</strong> {station['elevation']}m</p>
            <p><strong>State:</strong> {station['state']}</p>
        </div>
        """
        
        # Color based on elevation
        if station['elevation'] > 1000:
            color = 'red'
        elif station['elevation'] > 500:
            color = 'orange'
        else:
            color = 'green'
            
        folium.Marker([station['latitude'], station['longitude']], 
                     popup=popup_text, 
                     icon=folium.Icon(color=color)).add_to(m6)
    
    print(f"Added {len(stations_sample)} real weather stations to the map")
    m6
    
except Exception as e:
    print(f"Could not load real weather data: {e}")
    print("Using sample data instead...")
    
    # Fallback to sample data
    m6 = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
    for station in stations:
        folium.Marker([station['lat'], station['lon']], 
                     popup=f"{station['name']}: {station['temp']}°C").add_to(m6)
    m6


Could not load real weather data: 'DataFrame' object has no attribute 'compute'
Using sample data instead...


## Summary

This demo shows the key features of folium for weather data visualization:

1. **Basic Maps**: Simple map creation with markers
2. **Multiple Markers**: Adding multiple weather stations
3. **Color Coding**: Visual indicators based on data values
4. **Heatmaps**: Density visualization of temperature data
5. **Layers**: Multiple map layers and controls
6. **Real Data**: Integration with actual weather station data

### Key Features Demonstrated:
- Interactive markers with popups
- Color-coded visualization
- Heatmap overlays
- Multiple map layers
- Real weather data integration
- Custom styling and HTML popups

### Note: 
We switched from pyleaflet to folium because pyleaflet has very limited functionality and doesn't support markers or advanced features. Folium is a much more powerful and feature-rich library for interactive maps.
